In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup
import pandas as pd 
import time


In [2]:
edge_driver_path = "driverpath"

In [3]:
def sheet_connection():
    scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(r'path', scopes) #access the json key you downloaded earlier
    file = gspread.authorize(credentials) # authenticate the JSON key with gspread

    
    sheet = file.open("Influencers Contact Book")

    ws = sheet.get_worksheet(2)

    data = pd.DataFrame(ws.get_all_records())

#data , ws
    
    return data,ws

In [25]:
df,ws = sheet_connection()

In [27]:
df[df['FOLLOWERS'] == ''].shape

(72, 27)

In [23]:
def get_data():
    df,ws = sheet_connection()

    df_copy = df[615:]

    edge_options = EdgeOptions()

    edge_options.add_argument("start-maximized")
    edge_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    edge_options.add_experimental_option('useAutomationExtension', False)

    service = EdgeService(executable_path=edge_driver_path)
    driver = webdriver.Edge(service=service, options=edge_options)


    driver.get("https://socialblade.com")

    accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[6]/div/li[2]/a')))
    accept_button.click()

    time.sleep(1)


    username=driver.find_element(By.XPATH ,'/html/body/div[10]/div[2]/form/div[1]/input[1]')
    password=driver.find_element(By.XPATH ,'/html/body/div[10]/div[2]/form/div[2]/input')
    username.clear()
    password.clear()
    username.send_keys("dawnghanadi@gmail.com")
    password.send_keys("1357246")
    accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[10]/div[2]/form/div[3]/input')))
    accept_button.click()
    # Create an empty list to store the scraped data
    
    #getting blank rows
    #df_copy = df[df['FOLLOWERS'] == '']

    time.sleep(3)
    # Loop through the rows in the DataFrame
    for index, row in df_copy.iterrows():
        time.sleep(1)

        data = []
        print('Scraping {} Account...'.format(row['ID']))
        try:
            # Search for the ID
                accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div/div[2]/form/div[2]/div/div/span'))).click()
                time.sleep(2)
                accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div/div[2]/form/div[2]/div/ul/li[4]'))).click()

                search_input=driver.find_element(By.XPATH ,'/html/body/div[7]/div/div[2]/form/div[1]/input')
                search_input.clear()
                search_input.send_keys(row['ID'])
                accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div/div[2]/form/div[1]/button/i')))
                accept_button.click()

                time.sleep(4)
                # Scrape the data
                table = driver.find_element(By.XPATH , '/html/body/div[11]/div[2]/div/div[3]')
                first_box_data = table.text
                first_box_dict = {'MEDIA_UPLOADS':first_box_data.split('\n')[1] ,
                    'FOLLOWERS':first_box_data.split('\n')[3] , 
                    'FOLLOWING' : first_box_data.split('\n')[5],
                    'ENGAGEMENT_RATE' : first_box_data.split('\n')[8],
                    'AVG_LIKES' : first_box_data.split('\n')[10],
                    'AVG_COMMENTS' : first_box_data.split('\n')[12]}

                # table2 = driver.find_element(By.XPATH , '/html/body/div[12]/div[1]/div[3]')
                # second_box_data = table2.text
                # second_box_dict = {'last_30_days_followers':second_box_data.split('\n')[0].split(' ')[0] ,
                #                 'Last_30_days_follwers_raise':second_box_data.split('\n')[0].split(' ')[1] ,
                #                 'last_30_days_following':second_box_data.split('\n')[2].split(' ')[0],
                #                 'Last_30_days_following_raise':second_box_data.split('\n')[2].split(' ')[1],
                #                 #'last_30_days_media':second_box_data.split('\n')[4].split(' ')[0],
                #                 'last_30_days_media_raise':second_box_data.split('\n')[4].split(' ')[1]}

                # Combine the data into a DataFrame and append it to the list
                row_data = {**first_box_dict}
                row_data['ID'] = row['ID']
                data.append(row_data)
                ws.update('V{}:AA{}'.format(index+2 , index+2) ,[[data[0]['MEDIA_UPLOADS'],data[0]['FOLLOWERS'],data[0]['FOLLOWING'],data[0]['ENGAGEMENT_RATE'],data[0]['AVG_LIKES'],data[0]['AVG_COMMENTS']]])
                
        except:
                pass
    # Close the webdriver
    driver.quit()
    
    # Create a DataFrame from the list of scraped data
    #cc = pd.merge(df,pd.DataFrame(data),on = 'ID' , how = 'outer')


    # ws.append_row(data.values, table_range="V1:AA1")
    # for i in range(len(data)): 
    #     ws.update("V1:AA",[[data[i]]])

    return 





In [24]:
check = get_data()


Incompatible release of msedgedriver (version 114.0.1823.82) detected in PATH: c:\Users\S.Ghanadi\Desktop\inf_sheet\msedgedriver.exe


Scraping parmidamamaghani Account...
Scraping shaqayeq.mahmoodi Account...
Scraping ahooo_beautiful_angel Account...
Scraping tiamamirbakhtiar Account...


In [62]:
check.isnull().sum()

ID                               0
Eng May 2023                     0
Follower May 2023                0
Eng July 2023                    0
Follower July 2023               0
Name                             0
Link                             0
Phone Number                     0
Description                      0
Activity - 1                     0
Activity - 2                     0
Sex                              0
Marital status                   0
BackGround Check                 0
Social Media                     0
Phone                            0
City                             0
Price Post                       0
Off Pirce                        0
Price Story                      0
Off Story                        0
MEDIA_UPLOADS                   29
FOLLOWERS                       29
FOLLOWING                       29
ENGAGEMENT_RATE                 29
AVG_LIKES                       29
AVG_COMMENTS                    29
last_30_days_followers          29
Last_30_days_follwer